In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

from glob import glob

from tqdm import tqdm
import pykep as pk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.neighbors import NearestNeighbors

import h5py

from mpl_toolkits import mplot3d
%matplotlib notebook

dt = 10 #timestep of the inspected simulations, affects time in plots
iterations = 3153600 # for time computation as well

# PK epochs of simulation time
starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000')
end_t = pk.epoch(starting_t.mjd2000 + iterations * dt * pk.SEC2DAY)
total_days = end_t.mjd - starting_t.mjd

In [ ]:
data = h5py.File("../build/simulationData.h5")
iterations_idx = list(data["ParticleData"].keys())
iterations_idx = [int(it) for it in iterations_idx]
iterations_idx.sort()
iterations = max(iterations_idx)
print(iterations)

In [ ]:
rs,vs = [],[] #will hold r,v for whole simulation

# load vtks
for idx in iterations_idx:
    
    v = np.array(data["ParticleData"][str(idx)]["Particles"]["Velocities"])
    r = np.array(data["ParticleData"][str(idx)]["Particles"]["Positions"])
    rs.append(r)
    vs.append(v)

In [ ]:
# Threshold for conjunction tracking as used in simulation
thresholds = [1,5,10,20,25,50,75,100,250,500]

In [ ]:
# Convert conjunctions to pandas dataframe
conj = pd.DataFrame(columns=["P1","P2","Iteration","SquaredDistance"])
for it in tqdm(data["CollisionData"].keys()):
    iteration = int(it)
    collisions = np.array(data["CollisionData"][it]["Collisions"])
    for collision in collisions:
        conj = conj.append({"P1":collision[0], 
                     "P2": collision[1], 
                     "Iteration": iteration, 
                     "SquaredDistance": collision[2]},
                    ignore_index=True)

In [ ]:
conj

In [ ]:
# Sort ascending by conjunction distance, then drop all but the first to get closest encounter
conj = conj.sort_values('SquaredDistance', ascending=True)
unique_conj = conj.drop_duplicates(subset=["P1","P2"],keep="first")
unique_conj["Distance"] = np.sqrt(unique_conj.SquaredDistance) * 1000. # Compute distance in meters

In [ ]:
unique_conj

In [ ]:
# Compute conjunction counts at specific points in the simulation
stepsize = 100
# Time axis
t = np.linspace(0,end_t.mjd - starting_t.mjd,stepsize)
it = np.linspace(0,iterations,stepsize)
summed_conjs = [[] for _ in thresholds]
for i in tqdm(it):
    for idx,threshold in enumerate(thresholds):
        count = len(unique_conj[(unique_conj.Iteration < i) & (unique_conj.Distance < threshold)])
        summed_conjs[idx].append(count)

In [ ]:
fig = plt.figure(figsize=(9,5),dpi=100)
fig.patch.set_facecolor('white')

# Iterate over thresholds and plot for each
for idx,row in enumerate(thresholds):
    plt.plot(t,summed_conjs[idx],linewidth=3)
    
plt.legend([str(t) + "m" for t in thresholds],loc='upper center', bbox_to_anchor=(1.1,0.8), ncol=1, fancybox=True, shadow=True)
plt.title("Conjunction Thresholds Comparison")
plt.xlabel("Days")
plt.ylabel("# of Conjunctions")
plt.gca().set_yscale("log")
plt.tight_layout()

In [ ]:
# Compute conjunction counts at specific points in the simulation
steps = 1000
max_threshold = 1000
# Time axis
threshold_grid = np.logspace(-1,3,steps)
sums = []
for idx,threshold in enumerate(threshold_grid):
    count = len(unique_conj[(unique_conj.Distance < threshold)])
    sums.append(count)

fig = plt.figure(figsize=(5,5),dpi=100)
fig.patch.set_facecolor('white')

plt.plot(threshold_grid,sums,linewidth=3)
    
plt.title("Conjunction Thresholds vs. Conjunctions")
plt.xlabel("Threshold [m]")
plt.ylabel("# of Conjunctions")
# plt.gca().set_xscale("log")
# plt.gca().set_yscale("log")
plt.tight_layout()

In [ ]:
vtk_it = np.linspace(0,iterations,iterations+1)[::1200]
import math
def find_closest_it(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx
    else:
        return idx

In [ ]:
for row in conj.iterrows():
    idx = 
    r1,v1 = 
    print(row)

In [ ]:
# Compute min, max and mean of orbital elements over simulation
min_elements = [[],[],[],[],[],[]]
mean_elements = [[],[],[],[],[],[]]
max_elements = [[],[],[],[],[],[]]

stepsize = 2

for v_it,r_it in tqdm(zip(vs[::stepsize],rs[::stepsize]),total=len(vs[::stepsize])):
    elements = [[],[],[],[],[],[]]
    for v,r in zip(v_it,r_it):
        a,e,i,W,w,E = pk.ic2par(r,v, pk.MU_EARTH)
        elements[0].append(abs(a))
        elements[1].append(abs(e))
        elements[2].append(abs(i))
        elements[3].append(abs(W))
        elements[4].append(abs(w))
        elements[5].append(abs(E))
    for i in range(6):
        min_elements[i].append(np.min(elements[i]))
        mean_elements[i].append(np.mean(elements[i]))
        max_elements[i].append(np.max(elements[i]))

In [ ]:
#t ime axis of the simulation
t = np.linspace(0,end_t.mjd - starting_t.mjd,len(mean_elements[0]))

# Plot for each orbital element
for idx,element in enumerate(["a","e","i","W","w","E"]):
    fig = plt.figure(figsize=(6,4),dpi=100)
    fig.patch.set_facecolor('white')
    plt.plot(t,mean_elements[idx],linewidth=1)
#     plt.plot(t,min_elements[idx],linewidth=1)
#     plt.plot(t,max_elements[idx],linewidth=1)
#     plt.legend(["mean","min","max"],loc='upper center', bbox_to_anchor=(1.1,0.8), ncol=1, fancybox=True, shadow=True)
    plt.title("Evolution of "+element)
    plt.xlabel("Days")
    plt.ylabel("Mean " + element)
    plt.tight_layout()
#         plt.gca().set_yscale("log")

In [ ]:
# Compute a KNN to get distance to nearest neighbors over simulation

all_distances = []

r_subset = rs[::1]

for r_it in tqdm(r_subset,total=len(r_subset)):
    elements = [[],[],[],[],[],[]]
    knn = NearestNeighbors(n_neighbors=2).fit(r_it)
    distances,_ = knn.kneighbors(r_it)
    distances = distances[:,1] / 1000 # convert to km
    all_distances.append(distances)
    
all_distances = np.asarray(all_distances)

In [ ]:
# Subsample to look only at those below some threshold
small_distances = []
max_dist = 10
for dist in all_distances:
    small_distances.append(dist[dist < max_dist])

In [ ]:
# Compute distributions for each iteration
bins = 64
x = np.linspace(0, max_dist, bins)
y = np.linspace(0, total_days, len(all_distances))

X, Y = np.meshgrid(x, y)
Z = []
for dist in tqdm(small_distances):
    hist,bin_vals = np.histogram(dist, bins = x,density=False)
    hist = np.cumsum(hist)
    hist = np.concatenate([hist,[hist[-1]]]) # last value remains for bucket
    Z.append(hist)
x = np.concatenate([[0],x])
Z = np.asarray(Z)

In [ ]:
# Create beautiful plots
fig = plt.figure(figsize = (8,8),dpi=100)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z,  rstride=1, cstride=1, cmap="plasma", edgecolor='none')
ax.view_init(elev=20., azim=120)
ax.set_xlabel('Closest Distance [km]')
ax.set_ylabel('Days')
ax.set_zlabel('Cumulative Frequency');
ax.set_xlim([0,max_dist])